# Proyecto PixelSounds

In [1]:
import cv2
import os
import numpy as np
import warnings
import scipy.fftpack as fft
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d, median_filter
from scipy import signal
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    import vozyaudio as vz

## Transformada de Fourier en una imagen

In [2]:
def procesar_imagen_fft(ruta_imagen, fs, N=50,duracion=5):
    
    # Se lee y se aplana la imagen se pasa a gris para q los valores sean de 0 a 255
    imagen = cv2.imread(ruta_imagen, cv2.IMREAD_GRAYSCALE)
    imagen = cv2.resize(imagen, (256, 256))
    valores_pixeles = imagen.flatten()
    
    # Se aplica la transformada de fourier para obtener las frecuencias (gracias Victor)
    espectro = np.abs(fft.fft(valores_pixeles))
    freqs = fft.fftfreq(len(valores_pixeles), d=1/fs)
    
    # se eliminan las frecuencias negativas
    freqs = freqs[freqs > 0]
    espectro = espectro[:len(freqs)]
    
    # se sacan las frecuencias principales
    indices_principales = np.argsort(espectro)[-N:]
    freqs_principales = freqs[indices_principales]
    
    # Se genera la señal de audio, se hace asi por la funcion de voz 
    t = np.linspace(0, duracion, int(fs * duracion), endpoint=False)
    audio = np.zeros_like(t)
    
    for f in freqs_principales:
        audio += np.sin(2 * np.pi * f * t)
    
    # se normaliza la señal
    audio = audio / np.max(np.abs(audio))
    
    vz.sonido(audio, fs)
    
    plt.plot(t[:1000], audio[:1000],label=nombre_archivo)
    plt.title("Forma de onda generada")
    plt.xlabel("Tiempo")
    plt.ylabel("Amplitud")
    plt.legend()  

In [ ]:
# Creación de señales a partir de las componentes frecuenciales de la imagen

fs = 44100
for nombre_archivo in os.listdir("images"):
    if nombre_archivo.endswith(('.png', '.jpg', '.JPG')):
        ruta_imagen = os.path.join("images", nombre_archivo)
        procesar_imagen_fft(ruta_imagen, fs)

plt.show()